In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

import nltk
import unicodedata
from nltk.corpus import stopwords
import re
from itertools import chain
import unicodedata

In [2]:
raw_data = pd.read_csv('finaldata.csv')
raw_data.shape

C:\Users\anike\AppData\Roaming\Python\Python39\site-packages\IPython\core\interactiveshell.py:3418: DtypeWarning: Columns (7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


(31653, 52)

In [3]:
new_data = raw_data.iloc[:,[2,3,4,6]]
coloumns = ['retweets','replies','likes', 'text' ]
new_data.columns = coloumns

new_data.drop_duplicates(subset ="text",
                     keep = "first", inplace = True)
new_data = new_data.reset_index(drop=True)


c:\Python39\lib\site-packages\pandas\util\_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)


In [4]:
#LIST OF SENTENCES
sentences = new_data['text']
print(sentences)

0        text:"RT @harzior: Keep on smiling classic tee...
1        text:"Readers make leaders classic tee in as m...
2        text:"I love Reading classic tee in as many co...
3        text:"Votes for Women classic tee in as many c...
4        text:"Hawkins High School classic tee in as ma...
                               ...                        
16755                      text:"https://t.co/I30WP0VgnC"}
16756    text:"29/07/2022 20:00:01: XBT Bitmex Trollbox...
16757    text:"@aksysgames this is how I feel about Nor...
16758    text:"RT @RiCcONyC1: Workouts Be Startin Off W...
16759             text:"@girlgone_crypto Always a hoot 👍"}
Name: text, Length: 16760, dtype: object


In [5]:
%store -r unique_words
print(len(unique_words))

7522


In [6]:
def unicode_to_ascii(s):
    return ''.join(c for c in unicodedata.normalize('NFD', s) if unicodedata.category(c) != 'Mn')

def clean_stopwords_shortwords(w):
    stopwords_list=stopwords.words('english')
    words = w.split() 
    clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]

    clean_wordss = [word for word in clean_words if word in unique_words and len(word) > 2]
    unnecessary_words = ["text", "fashion",'feel', 'always', 'mirror' "sale","art","dear","car", "love", "beauty", "style", "brand", "holiday", "new", "good", "check","self"]
    #good_words = ["men", "women", "man", "woman", "boys", "girls", "male", "female"]
    clean_wordsss = [word for word in clean_wordss if (word not in unnecessary_words)]
    #clean_words = [word for word in words if (word not in stopwords_list) and len(word) > 2]
    #clean_words = [word for word in words if (word in good_words) and len(word)>2]
    
    
    return " ".join(clean_wordsss) 

def preprocess_sentence(w):
    w = unicode_to_ascii(w.lower().strip())
    w = re.sub(r"([?.!,¿])", r" ", w)
    w = re.sub(r'[" "]+', " ", w)
    w = re.sub(r"[^a-zA-Z?.!,¿]+", " ", w)
    w = re.sub(r"text+","",w) #removing "text" from every sentence
    w = re.sub(r"https+","",w)
    w=clean_stopwords_shortwords(w)
    w=re.sub(r'@\w+', '',w)
    return w

In [7]:
sentences= sentences.astype(str)
good_sentences = sentences.map(preprocess_sentence)
new_data['text'] = good_sentences

In [8]:
#DELETING EMPTY AND DUPLICATE ROWS

from more_itertools import first
count = 0
for i in range(0, len(sentences),1):
    if (good_sentences[i]==''):
        count+=1
        new_data = new_data.drop(labels=i, axis=0)  
             
new_data = new_data.reset_index(drop=True)
new_data.head()

,retweets,replies,likes,text
0,1.0,490.0,469.0,keep smiling classic tee colors possible men w...
1,0.0,393.0,500.0,make classic tee colors possible men women shi...
2,0.0,443.0,343.0,classic tee colors possible men women shirts p...
3,0.0,424.0,406.0,women classic tee colors possible men women sh...
4,0.0,354.0,460.0,high school classic tee colors possible men wo...


In [9]:
new_data

,retweets,replies,likes,text
0,1.0,490.0,469.0,keep smiling classic tee colors possible men w...
1,0.0,393.0,500.0,make classic tee colors possible men women shi...
2,0.0,443.0,343.0,classic tee colors possible men women shirts p...
3,0.0,424.0,406.0,women classic tee colors possible men women sh...
4,0.0,354.0,460.0,high school classic tee colors possible men wo...
...,...,...,...,...
11747,0.0,191.0,277.0,try essential blues shorts
11748,0.0,244.0,221.0,ball play sports hannah
11749,12.0,266.0,181.0,red compression shorts
11750,0.0,291.0,253.0,long


In [10]:
good_sentences = new_data['text']
%store good_sentences
%store new_data

Stored 'good_sentences' (Series)
Stored 'new_data' (DataFrame)


In [11]:
good_sentences

0        keep smiling classic tee colors possible men w...
1        make classic tee colors possible men women shi...
2        classic tee colors possible men women shirts p...
3        women classic tee colors possible men women sh...
4        high school classic tee colors possible men wo...
                               ...                        
11747                           try essential blues shorts
11748                              ball play sports hannah
11749                               red compression shorts
11750                                                 long
11751                                        mirror shorts
Name: text, Length: 11752, dtype: object